In [43]:
from elasticsearch import Elasticsearch
import os
import requests
from requests.utils import quote
import json

API_KEY = "2oYO3lQwGjIr9RgV+oRa6w==LsM3nPTL7VMFT7m5"
NURT_API_ID = "73bd8bbc"
NURT_API_KEY = "46ff5210638f6e9f5c84ee68eb32033f"

In [30]:
# Get Elasticsearch host from environment variables
elasticsearch_host = "http://localhost:9200"
elasticsearch_username = "elastic"
elasticsearch_password = "p5FE3c=alPhGd20o14bx"
# Connect to Elasticsearch
try:
    es = Elasticsearch(elasticsearch_host, basic_auth=(elasticsearch_username, elasticsearch_password))
    if es.ping():
        print("Successfully connected to Elasticsearch!")
    else:
        print("Failed to connect to Elasticsearch.")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

Successfully connected to Elasticsearch!


In [31]:
def fetch_recipes(query):
    """Fetch recipes from API-Ninjas and return JSON response."""
    api_url = f"https://api.api-ninjas.com/v1/recipe?query={query}"
    headers = {"X-Api-Key": API_KEY}
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []

def store_recipes_in_elasticsearch(recipes):
    """Store the fetched recipes in Elasticsearch."""
    for i, recipe in enumerate(recipes):
        es.index(index="recipes", id=i+1, document=recipe)
    print("Recipes successfully stored in Elasticsearch!")

def search_recipes(keyword):
    """Search for recipes in Elasticsearch."""
    results = es.search(index="recipes", query={
        "match": {
            "title": keyword  # Searches in the recipe title field
        }
    })
    return results["hits"]["hits"]

In [32]:
# Example: Fetch & Store "Italian Wedding Soup"
query = "italian wedding soup"
recipes = fetch_recipes(query)

if recipes:
    store_recipes_in_elasticsearch(recipes)

Recipes successfully stored in Elasticsearch!


In [33]:
# Example search
search_results = search_recipes("soup")

for recipe in search_results:
    print(recipe["_source"])

{'title': 'Italian Wedding Soup', 'ingredients': '1/2 lb Ground beef|1/2 lb Ground veal|1/4 c Italian seasoned bread crumb|1 Egg|1 tb Parsley|Salt and pepper to taste|4 c Chicken broth|2 c Spinach leaves cut into piec|1/4 c Grated Pecorino Romano chees', 'servings': '1 Servings', 'instructions': 'Combine the ground meat, bread crumbs, egg, parsley, salt and pepper in a bowl. Mix well and form into tiny meat balls. Bake on a cookie sheet for 30 minutes at 350F. Meanwhile, bring broth to a boil and add spinach. Cover and boil for 5 minutes. Add the meatballs to the hot broth, bring to a simmer. Stir in the cheese and serve immediately. Rita in Scottsdale 01/02/92 01:41 am'}
{'title': 'Italian Wedding Soup', 'ingredients': '2 qt Chicken stock|1 Chopped carrot|1/2 Chopped onion|1 Chopped celery|2 oz Ground meat; (or ground vegieburger can be used)|1 Egg|1 Sprig of chopped parsely', 'servings': '4 - 6 servin', 'instructions': 'Bring chicken stock to a boil add the chopped carrot,celery and 

In [47]:
def fetch_nutrition(query):
    """Fetch recipes from Nutritionix and return JSON response."""
    # URL encode the query to handle special characters
    encoded_query = quote(query)
    api_url = f"https://trackapi.nutritionix.com/v2/search/instant/?query={encoded_query}"
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': NURT_API_ID,
        'x-app-key': NURT_API_KEY       
    }
    
    response = requests.get(api_url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []
    
def fetch_nutrition_NL(query):  
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        'Content-Type': 'application/json',
        'x-app-id': NURT_API_ID,
        'x-app-key': NURT_API_KEY
    }

    # Set up the body of the request
    body = {
        "query": query  
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))

    # Check for errors
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")

In [48]:
results = fetch_nutrition_NL("apple")
print(results['foods'])

[{'food_name': 'apple', 'brand_name': None, 'serving_qty': 1, 'serving_unit': 'medium (3" dia)', 'serving_weight_grams': 182, 'nf_calories': 94.64, 'nf_total_fat': 0.31, 'nf_saturated_fat': 0.05, 'nf_cholesterol': 0, 'nf_sodium': 1.82, 'nf_total_carbohydrate': 25.13, 'nf_dietary_fiber': 4.37, 'nf_sugars': 18.91, 'nf_protein': 0.47, 'nf_potassium': 194.74, 'nf_p': 20.02, 'full_nutrients': [{'attr_id': 203, 'value': 0.4732}, {'attr_id': 204, 'value': 0.3094}, {'attr_id': 205, 'value': 25.1342}, {'attr_id': 207, 'value': 0.3458}, {'attr_id': 208, 'value': 94.64}, {'attr_id': 209, 'value': 0.091}, {'attr_id': 210, 'value': 3.7674}, {'attr_id': 211, 'value': 4.4226}, {'attr_id': 212, 'value': 10.738}, {'attr_id': 213, 'value': 0}, {'attr_id': 214, 'value': 0}, {'attr_id': 221, 'value': 0}, {'attr_id': 255, 'value': 155.7192}, {'attr_id': 262, 'value': 0}, {'attr_id': 263, 'value': 0}, {'attr_id': 268, 'value': 396.76}, {'attr_id': 269, 'value': 18.9098}, {'attr_id': 287, 'value': 0}, {'attr